In [26]:
import numpy as np
import cupy as cp
import time

def validate_matrix_dims(mat1, mat2):
    """Проверка, что умножение матриц возможно."""
    if mat1.shape[1] != mat2.shape[0]:
        raise ValueError("Размеры матриц не соответствуют правилам умножения")

def cpu_matrix_mult(mat1, mat2):
    """Умножение матриц на CPU."""
    validate_matrix_dims(mat1, mat2)  # Вызов проверки совместимости

    output = np.zeros((mat1.shape[0], mat2.shape[1]))

    start = time.time()
    for row in range(mat1.shape[0]):
        for col in range(mat2.shape[1]):
            for index in range(mat1.shape[1]):
                output[row, col] += mat1[row, index] * mat2[index, col]
    end = time.time()

    return output, end - start

def gpu_matrix_mult(mat1, mat2):
    """Умножение матриц на GPU."""
    validate_matrix_dims(mat1, mat2)  # Вызов проверки совместимости

    # Перенос данных матриц на графическую карту
    mat1_gpu = cp.asarray(mat1)
    mat2_gpu = cp.asarray(mat2)

    start = time.time()
    result_gpu = cp.dot(mat1_gpu, mat2_gpu)  # Быстрое умножение на GPU
    final_result = cp.asnumpy(result_gpu)    # Перенос результата обратно на CPU
    end = time.time()

    return final_result, end - start

if __name__ == "__main__":
    shape = (800, 800)
    mat1 = np.random.random(shape)
    mat2 = np.random.random(shape)

    cpu_result, cpu_exec_time = cpu_matrix_mult(mat1, mat2)
    gpu_result, gpu_exec_time = gpu_matrix_mult(mat1, mat2)

    print(f"Результат на CPU:\n{cpu_result}")
    print(f"Время выполнения на CPU: {cpu_exec_time:.6f} сек")

    print(f"Результат на GPU:\n{gpu_result}")
    print(f"Время выполнения на GPU: {gpu_exec_time:.6f} сек")

Результат на CPU:
[[196.64866019 199.01542616 207.22579922 ... 197.89240849 204.92230917
  198.37771721]
 [200.27804083 197.15458353 207.04532163 ... 198.00839405 202.45500537
  200.38637497]
 [192.71463242 195.81326324 197.0890396  ... 192.06756078 197.09128588
  199.86879558]
 ...
 [199.849455   202.44288754 210.84737945 ... 199.60570073 207.05058735
  208.45043831]
 [192.87384471 193.27951379 198.96042739 ... 190.87206161 196.86199943
  200.54598396]
 [199.99056972 199.96146378 210.05213696 ... 199.9049639  201.62555599
  203.20149569]]
Время выполнения на CPU: 272.139719 сек
Результат на GPU:
[[196.64866019 199.01542616 207.22579922 ... 197.89240849 204.92230917
  198.37771721]
 [200.27804083 197.15458353 207.04532163 ... 198.00839405 202.45500537
  200.38637497]
 [192.71463242 195.81326324 197.0890396  ... 192.06756078 197.09128588
  199.86879558]
 ...
 [199.849455   202.44288754 210.84737945 ... 199.60570073 207.05058735
  208.45043831]
 [192.87384471 193.27951379 198.96042739 ..